## Abschlussproket
# Einführung in Python für Data Analytics
### Dozent: Dr. Alessandro Bramucci
### Bearbeitung und Präsentation: Jonathan, Gregor, Sebastian

#### Import Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

#### Import data

In [4]:
# speichere den relativen Pfad und den Dateinamen der Quelle 
file_besucher = 'data/besucher.csv'
file_geo = 'data/geo.txt'
file_kunden = 'data/kunden.csv'

# importiere Daten aus der Quelle in Dataframes
df_besucher = pd.read_csv(file_besucher, sep=";", decimal = ',')
df_geo = pd.read_table(file_geo, decimal = '.')
df_kunden = pd.read_csv(file_kunden, sep=";", decimal = '.')

In [6]:
#df_kunden['Geschlecht'] = df_kunden.to_integer(df_kunden['Geschlecht'])
df_kunden['Geschlecht'].fillna(2, inplace = True) 
df_kunden['Geschlecht'] = df_kunden['Geschlecht'].astype(int)

In [7]:
for x in df_geo.index:
    lower_niederlassung = df_geo.loc[x, 'Niederlassung'].lower()
    if "berlin" in df_geo.loc[x, 'Niederlassung']:
        print(df_geo.loc[x, 'Niederlassung'])
        df_geo.loc[x, 'Niederlassung'] = "Berlin"
        print(df_geo.loc[x, 'Niederlassung'])

In [15]:
df_geo['Niederlassung'] = df_geo['Niederlassung'].apply(lambda x: 'Nordrhein-Westfalen' if 'NRW' in x else x)
df_geo['Niederlassung'] = df_geo['Niederlassung'].apply(lambda x: 'Berlin' if 'Berlin' in x else x)

In [16]:
df_besucher.dtypes

Alter           int64
Einkommen     float64
Geschlecht      int64
Zeit          float64
KundeNr        object
dtype: object

In [17]:
df_besucher.head(5)

,Alter,Einkommen,Geschlecht,Zeit,KundeNr
0,30,39919.095335,0,29.0,K1814
1,41,53430.673086,0,27.0,K1544
2,64,73163.216927,0,32.0,K1945
3,41,36761.482581,0,44.0,K1825
4,35,42684.331632,1,33.0,K1144


In [18]:
df_geo.dtypes

KundeNr          object
Niederlassung    object
dtype: object

In [19]:
df_geo.head(5)

,KundeNr,Niederlassung
0,K0001,Sachsen
1,K0002,Nordrhein-Westfalen
2,K0003,Hessen
3,K0004,Bayern
4,K0005,Bayern


In [20]:
df_kunden.dtypes

Alter           int64
Einkommen     float64
Preis         float64
Geschlecht      int64
Zeit          float64
KundeNr        object
dtype: object

In [21]:
df_kunden.head(5)

,Alter,Einkommen,Preis,Geschlecht,Zeit,KundeNr
0,64,66894.0,88160.31,1,43.0,K0310
1,54,77644.0,103145.70,1,40.0,K1042
2,55,44341.0,80565.16,0,37.0,K0382
3,49,67271.0,83949.89,0,42.0,K0498
4,46,49832.0,93781.58,0,41.0,K0552
